In [22]:
### https://carpentries-incubator.github.io/gpu-speedups/01_CuPy_and_Numba_on_the_GPU/index.html
#### NUMBA CONFLICT WITH PYTORCH
#### OSError: [WinError 182] The operating system cannot run %1. Error loading 
#### "C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\torch\lib\caffe2_detectron_ops.dll" or one of its dependencies.
##### =====> conda install intel-openmp --force-reinstall
import numpy as np
import cupy as cp

memory_pool = cp.cuda.MemoryPool()
cp.cuda.set_allocator(memory_pool.malloc)
pinned_memory_pool = cp.cuda.PinnedMemoryPool()
cp.cuda.set_pinned_memory_allocator(pinned_memory_pool.malloc)

import cupyx.scipy.ndimage as ndimage_cp
import scipy.ndimage as ndimage_np
from cupyx.profiler import benchmark
import scipy.ndimage as ndimage_np
image1 = np.random.rand(100,1000,1000).astype(np.float32)
image2 = cp.random.rand(100,1000,1000).astype(cp.float32)

def my_func(a):
    return ndimage_cp.zoom(a, (1, 0.1, 0.1))

%timeit ndimage_np.zoom(image1, (1, 0.1, 0.1))
print(benchmark(my_func, (image2,), n_repeat=20))  

3.3 s ± 41.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
my_func             :    CPU:  531.215 us   +/-98.008 (min:  483.600 / max:  950.500) us     GPU-0:29382.161 us   +/-346.140 (min:28899.328 / max:30443.520) us


In [23]:
import math  # Note that for the CUDA target, we need to use the scalar functions from the math module, not NumPy
import numpy as np
from numba import vectorize

@vectorize(['int64(int64, int64)'], target='cuda')
def add_ufunc(x, y):
    return x + y

SQRT_2PI = np.float32((2*math.pi)**0.5)  # Precompute this constant as a float32.  Numba will inline it at compile time.

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def gaussian_pdf(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)


In [24]:
# Evaluate the Gaussian distribution PDF a million times!
x = np.random.uniform(-300, 300, size=100000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)

# Quick test
gaussian_pdf(x[0], 0.0, 1.0)

C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


array([7.73574874e-224])

In [18]:
import scipy.stats # for definition of gaussian distribution
norm_pdf = scipy.stats.norm
%timeit norm_pdf.pdf(x, loc=mean, scale=sigma)

5.83 ms ± 66.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%timeit gaussian_pdf(x, mean, sigma)

C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (131) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


1.97 ms ± 54.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
from numba import cuda

@cuda.jit(device=True)
def polar_to_cartesian(rho, theta):
    x = rho * math.cos(theta)
    y = rho * math.sin(theta)
    return x, y  # This is Python, so let's return a tuple

@vectorize(['float32(float32, float32, float32, float32)'], target='cuda')
def polar_distance(rho1, theta1, rho2, theta2):
    x1, y1 = polar_to_cartesian(rho1, theta1)
    x2, y2 = polar_to_cartesian(rho2, theta2)
    
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

In [21]:
n = 1000000
rho1 = np.random.uniform(0.5, 1.5, size=n).astype(np.float32)
theta1 = np.random.uniform(-np.pi, np.pi, size=n).astype(np.float32)
rho2 = np.random.uniform(0.5, 1.5, size=n).astype(np.float32)
theta2 = np.random.uniform(-np.pi, np.pi, size=n).astype(np.float32)

In [23]:
polar_distance(rho1, theta1, rho2, theta2)

array([0.4676987 , 0.81071669, 1.73532505, ..., 1.38706656, 1.33927193,
       1.50384252])

In [21]:
import numba

@numba.jit
def polar_to_cartesian_cpu(rho, theta):
    x = rho * math.cos(theta)
    y = rho * math.sin(theta)
    return x, y  # This is Python, so let's return a tuple

@vectorize(['float32(float32, float32, float32, float32)'])  # default target is CPU
def polar_distance_cpu(rho1, theta1, rho2, theta2):
    x1, y1 = polar_to_cartesian_cpu(rho1, theta1)
    x2, y2 = polar_to_cartesian_cpu(rho2, theta2)
    
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

np.testing.assert_allclose(polar_distance(rho1, theta1, rho2, theta2),
                           polar_distance_cpu(rho1, theta1, rho2, theta2),
                           rtol=1e-7, atol=5e-7)

In [24]:
%timeit polar_distance_cpu(rho1, theta1, rho2, theta2)
%timeit polar_distance(rho1, theta1, rho2, theta2)

42.6 ms ± 728 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
11.4 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


CuPy is NumPy, but for the GPU

Data is copied from the CPU (host) to the GPU (device), where it is computed on. After a computation, it need to be copied back to the CPU to be interacted with by numpy, etc

%timeit can be used to benchmark the runtime of GPU spedup functions

GPU spedup functions are optimized for at least four things: 1. input size 2. compute complexity 3. CPU/GPU copying 4. data type. Concretely, a gpu spedup function can be slow because the input size is too small, the computation is too simple, there is excessive data copying to/from GPU/CPU, and the input types are excessivly large (e.g. np.float64 vs np.float32)

Make GPU spedup ufuncs with @numba.vectorize(..., target='cuda')

Make CUDA device functions with @numba.cuda.jit(device=True)

In [15]:
## =====> https://numba.readthedocs.io/en/stable/cuda/examples.html#vector-addition

In [5]:
import time
import cupy as cp
import numpy as np

s = time.time()
x_cpu = np.ones((1000,1000,100))
e = time.time()
print(e - s)### CuPy and GPU
s = time.time()
x_gpu = cp.ones((1000,1000,100))
e = time.time()
print(e - s)

0.12050390243530273
0.19729042053222656


In [8]:
from numba import cuda
import numpy as np

@cuda.jit
def max_example(result, values):
    """Find the maximum value in values and store in result[0]"""
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = (bid * bdim) + tid
    cuda.atomic.max(result, 0, values[i])


arr = np.random.rand(16384)
result = np.zeros(1, dtype=np.float64)

max_example[256,64](result, arr)
print(result[0]) # Found using cuda.atomic.max
print(max(arr))  # Print max(arr) for comparison (should be equal!)

0.9999952339622552
0.9999952339622552


C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [9]:
@cuda.jit
def max_example_3d(result, values):
    """
    Find the maximum value in values and store in result[0].
    Both result and values are 3d arrays.
    """
    i, j, k = cuda.grid(3)
    # Atomically store to result[0,1,2] from values[i, j, k]
    cuda.atomic.max(result, (0, 1, 2), values[i, j, k])

arr = np.random.rand(1000).reshape(10,10,10)
result = np.zeros((3, 3, 3), dtype=np.float64)
max_example_3d[(2, 2, 2), (5, 5, 5)](result, arr)
print(result[0, 1, 2], '==', np.max(arr))

0.9980142767371903 == 0.9980142767371903


C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (8) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [10]:
import numpy as np
from numba import cuda

In [12]:
@cuda.jit
def f(a, b, c):
    # like threadIdx.x + (blockIdx.x * blockDim.x)
    tid = cuda.grid(1)
    size = len(c)

    if tid < size:
        c[tid] = a[tid] + b[tid]

In [13]:
N = 100000
a = cuda.to_device(np.random.random(N))
b = cuda.to_device(np.random.random(N))
c = cuda.device_array_like(a)

In [14]:
f.forall(len(a))(a, b, c)
print(c.copy_to_host())

[1.2577078  1.42501432 0.67338825 ... 0.70452543 0.79853172 1.12594757]


C:\Users\rob\anaconda3\envs\spacy-GPU-env\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (131) < 2 * SM count (164) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
